In [14]:
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import warnings
import re
import ast

warnings.filterwarnings('ignore')


In [15]:
lift_df = pd.read_csv("lifts.csv", index_col=0)
data_for_lift = pd.read_csv("data_for_lift.csv", index_col=0)

In [17]:
lift_df

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>T')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'G>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '5', 'TERT', 'G>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '12', 'KRAS', 'C>T')",0.00
...,...,...,...
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.166666667, '25398284.0-25398284.0', 12.0, '...",0.00
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.233333333, '25398284.0-25398284.0', 12.0, '...",0.63
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.1, '55259515.0-55259515.0', 858.0, 'cTg/cGg...",0.00


In [17]:
# try_df = data_for_lift[data_for_lift["Cancer Type"] == "Colorectal Carcinoma"]
# try_df = data_for_lift[data_for_lift['Smoke Status'] == "Unknown"]
# try_df = try_df[try_df['SNP_event'] == "C>G"]
# try_df = try_df[try_df['Exon_Number'] == "02/05"]
# try_df = try_df[try_df['Protein_position'] == 12.0]
# try_df = try_df[try_df['Codons'] == "Ggt/Cgt"]
#
# set(try_df.index)
# try_df

,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,...,Extrahepatic Cholangiocarcinoma,Gallbladder Carcinoma,Gastric Cancer,Intrahepatic Cholangiocarcinoma,Liver Hepatocellular Carcinoma,Non Small Cell Lung Cancer,Pancreatic Cancer,Small Cell Lung Cancer,Soft Tissue Sarcoma,Position
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
Patient0257,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,61-70,Unknown,0.133333,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient0295,Non Small Cell Lung Cancer,Large Cell Lung Carcinoma,IV,Metastasis,Male,51-60,Unknown,1.866667,KRAS,12,...,0,0,0,0,0,1,0,0,0,25398285.0-25398285.0
Patient0378,Intrahepatic Cholangiocarcinoma,Intrahepatic Cholangiocarcinoma,Unknown,Primary,Male,51-60,Unknown,0.233333,KRAS,12,...,0,0,0,1,0,0,0,0,0,25398285.0-25398285.0
Patient0557,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,71-80,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient0582,Breast Carcinoma,Breast Invasive Carcinoma,IV,Metastasis,Female,41-50,Unknown,1.200000,KRAS,12,...,0,0,0,0,0,0,0,0,0,25398285.0-25398285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient8524,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Male,51-60,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient8538,Pancreatic Cancer,Pancreatic Adenocarcinoma,I,Primary,Male,41-50,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient8546,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Female,61-70,Unknown,0.100000,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0


In [18]:
lift_df.sort_values(by="Lift Value", ascending=False).head(1000)

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', 12.0, 'Ggt/Cgt')",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'Consequence',...","('Unknown', 'KRAS', 'missense_variant', 12.0, ...",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', '25398285.0-2539828...",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', 'Ggt/Cgt', 'Substit...",27.03
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', '25398285.0-25398285....",27.03
...,...,...,...
Pancreatic Cancer,"('Smoke Status', 'Consequence', 'Exon_Number',...","('Unknown', 'missense_variant', '02/05', '51-6...",16.64
Pancreatic Cancer,"('Chromosome', 'Hugo_Symbol', 'Exon_Number', '...","('12', 'KRAS', '02/05', 0.133333333, 12.0)",16.64
Pancreatic Cancer,"('Smoke Status', 'SNP_event', 'Diagnosis Age',...","('Unknown', 'C>A', '51-60', '25398284.0-253982...",16.64


In [19]:
df = lift_df.copy()

In [38]:
df.reset_index(inplace=True)

In [39]:
df.sort_values(by="Lift Value", ascending=False)

,index,Cancer Type,Feature Combination,Feature,Lift Value
12652,12652,Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '12', 'KRAS', 'C>G', 'missense_var...",27.03
717,717,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'A>G', 'm...",26.22
25,25,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'A>G', 'm...",26.22
69,69,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'missense...",23.73
159,159,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', '21/21', ...",23.73
...,...,...,...,...,...
6283,6283,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '4', 'FAT1', 'Truncation')",0.00
6278,6278,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '16', 'PLCG2', 'Substituti...",0.00
6268,6268,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Smoker', '8', 'FAM135B', 'Substituti...",0.00
6262,6262,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '10', 'RET', 'Substitution...",0.00


In [40]:
# Improved function to safely parse tuple-like strings
def safe_eval(val):
    if pd.isna(val) or val == 'None' or val == 'nan':
        return None

    try:
        # Handle the case where it's already a proper string representation of a tuple
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        try:
            # Fix missing quotes around words and try again
            fixed_val = re.sub(r'(\b\w+\b)(?=\s*,|\s*\))', r"'\1'", val)
            fixed_val = re.sub(r'\(\s*,', '(', fixed_val)  # Fix empty first item
            fixed_val = re.sub(r',\s*\)', ')', fixed_val)  # Fix empty last item
            return ast.literal_eval(fixed_val)
        except (ValueError, SyntaxError):
            # If still fails, try to extract content between parentheses
            match = re.search(r'\((.*)\)', val)
            if match:
                items = match.group(1).split(',')
                # Clean and convert items
                processed_items = []
                for item in items:
                    item = item.strip()
                    if item and item != 'None':
                        # Try to convert to appropriate type
                        try:
                            if item.lower() == 'true':
                                processed_items.append(True)
                            elif item.lower() == 'false':
                                processed_items.append(False)
                            elif item.replace('.', '', 1).isdigit():
                                if '.' in item:
                                    processed_items.append(float(item))
                                else:
                                    processed_items.append(int(item))
                            else:
                                processed_items.append(item.strip("'\""))
                        except:
                            processed_items.append(item.strip("'\""))

                return tuple(processed_items) if processed_items else None
            return None

# Apply parsing with tqdm progress bar
tqdm.pandas(desc="Parsing Features")
df["Feature"] = df["Feature"].progress_apply(lambda x: safe_eval(str(x)) if not pd.isna(x) else None)
df["Feature Combination"] = df["Feature Combination"].progress_apply(lambda x: safe_eval(str(x)) if not pd.isna(x) else None)

Parsing Features: 100%|██████████| 15864/15864 [00:00<00:00, 147014.11it/s]


In [41]:
# Find all unique features across the dataset
all_features = set()
for features in df["Feature Combination"].dropna():
    if isinstance(features, tuple):  # Make sure it's actually a tuple
        all_features.update(features)

# Prepare transformed data
expanded_data = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    new_row = {
        "Cancer Type": row["Cancer Type"],
        "Lift Value": row["Lift Value"]
    }

    # The bug is here: Check if both are tuples AND verify they're the same length
    if isinstance(row["Feature Combination"], tuple) and isinstance(row["Feature"], tuple):
        if len(row["Feature Combination"]) == len(row["Feature"]):
            new_row.update(dict(zip(row["Feature Combination"], row["Feature"])))
        else:
            # Handle case where tuples are different lengths
            # Only use the matching portions
            min_length = min(len(row["Feature Combination"]), len(row["Feature"]))
            new_row.update(dict(zip(row["Feature Combination"][:min_length],
                                    row["Feature"][:min_length])))

    # Assign NaN to missing feature columns
    for feature in all_features:
        if feature not in new_row:
            new_row[feature] = None  # Using None to represent NaN

    # Only append rows that have at least one non-null feature value
    if any(new_row.get(feature) is not None for feature in all_features):
        expanded_data.append(new_row)
    else:
        # Optionally include even empty rows if that's what you want
        expanded_data.append(new_row)

# Convert to DataFrame
df_transformed = pd.DataFrame(expanded_data)

Processing Rows: 100%|██████████| 15864/15864 [00:00<00:00, 65385.56it/s]


In [42]:
df_transformed

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous)
0,Breast Carcinoma,0.00,Male,Unknown,12,KRAS,C>T,missense_variant,02/05,61-70,25398284.0-25398284.0,12.0,gGt/gAt,Substitution/Indel,NaN
1,Breast Carcinoma,0.00,Female,Nonsmoker,7,EGFR,T>G,missense_variant,21/28,61-70,55259515.0-55259515.0,858.0,cTg/cGg,Substitution/Indel,NaN
2,Breast Carcinoma,0.00,Female,Unknown,12,KRAS,C>T,missense_variant,02/05,61-70,25398284.0-25398284.0,12.0,gGt/gAt,Substitution/Indel,NaN
3,Breast Carcinoma,0.00,Female,Unknown,12,KRAS,C>T,missense_variant,02/05,51-60,25398284.0-25398284.0,12.0,gGt/gAt,Substitution/Indel,NaN
4,Breast Carcinoma,0.00,Female,Nonsmoker,7,EGFR,T>G,missense_variant,21/28,51-60,55259515.0-55259515.0,858.0,cTg/cGg,Substitution/Indel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15859,Soft Tissue Sarcoma,1.73,Male,Unknown,None,None,None,frameshift_variant,None,51-60,None,NaN,None,Truncation,0.133333
15860,Soft Tissue Sarcoma,0.00,Male,Unknown,None,None,None,frameshift_variant,None,51-60,None,NaN,None,Truncation,0.266667
15861,Soft Tissue Sarcoma,0.62,Male,Unknown,None,None,None,stop_gained,None,61-70,None,NaN,None,Truncation,0.133333
15862,Soft Tissue Sarcoma,0.65,Male,Unknown,None,None,None,frameshift_variant,None,61-70,None,NaN,None,Truncation,0.133333


In [43]:
def merge_rows(df):
    # Step 1: First do the original grouping by exact Cancer Type and Lift Value
    grouped = df.groupby(['Cancer Type', 'Lift Value'])

    initial_merged_rows = []

    for (cancer_type, lift_value), group in grouped:
        # Start with the first row as the merged row
        merged_row = group.iloc[0].copy()

        # Go through all columns except 'Cancer Type' and 'Lift Value'
        for col in df.columns:
            if col not in ['Cancer Type', 'Lift Value']:
                # Fill in values from other rows in the group
                for i in range(1, len(group)):
                    current_val = merged_row[col]
                    new_val = group.iloc[i][col]

                    # If current value is NaN and new value is not, use new value
                    if pd.isna(current_val) and not pd.isna(new_val):
                        merged_row[col] = new_val

        initial_merged_rows.append(merged_row)

    # Step 2: Create a dataframe from the initial merging
    initial_merged_df = pd.DataFrame(initial_merged_rows)

    # Step 3: Now merge rows with lift value differences < 1
    # Sort by Cancer Type and Lift Value
    df_sorted = initial_merged_df.sort_values(['Cancer Type', 'Lift Value']).reset_index(drop=True)

    final_result_rows = []
    processed_indices = set()

    for i in range(len(df_sorted)):
        if i in processed_indices:
            continue

        cancer_type = df_sorted.iloc[i]['Cancer Type']
        lift_value = df_sorted.iloc[i]['Lift Value']

        # Find all rows with same cancer type and lift value within threshold
        similar_rows = []
        for j in range(len(df_sorted)):
            if j in processed_indices:
                continue

            if (df_sorted.iloc[j]['Cancer Type'] == cancer_type and
                abs(df_sorted.iloc[j]['Lift Value'] - lift_value) < 3):
                similar_rows.append(j)
                processed_indices.add(j)

        # Create a group DataFrame
        group = df_sorted.iloc[similar_rows]

        # Start with the first row as the merged row
        merged_row = group.iloc[0].copy()

        # Go through all columns except 'Cancer Type' and 'Lift Value'
        for col in df_sorted.columns:
            if col not in ['Cancer Type', 'Lift Value']:
                # Fill in values from other rows in the group
                for i in range(1, len(group)):
                    current_val = merged_row[col]
                    new_val = group.iloc[i][col]

                    # If current value is NaN and new value is not, use new value
                    if pd.isna(current_val) and not pd.isna(new_val):
                        merged_row[col] = new_val

        final_result_rows.append(merged_row)

    return pd.DataFrame(final_result_rows)

# Apply the function to your dataframe
merged_df = merge_rows(df_transformed)

In [44]:
merged_df.sort_values(by=["Hugo_Symbol", "Lift Value", "Cancer Type"], ascending=[True, False, True])

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous)
219,Breast Carcinoma,9.34,Female,Unknown,2,ALK,C>T,missense_variant,10/21,41-50,None,542.0,None,Substitution/Indel,0.133333
472,Colorectal Carcinoma,3.00,Female,Unknown,5,APC,C>T,stop_gained,16/16,51-60,112175639.0-112175639.0,1450.0,Cga/Tga,Truncation,NaN
1428,Gallbladder Carcinoma,12.42,Female,Unknown,12,ARID2,None,None,None,None,None,NaN,None,Truncation,NaN
1688,Gastric Cancer,6.18,Female,Unknown,16,CDH1,None,None,None,None,None,NaN,None,Substitution/Indel,NaN
2827,Pancreatic Cancer,12.32,Female,Unknown,9,CDKN2A,C>T,missense_variant,02/05,51-60,25398284.0-25398284.0,12.0,gGt/gAt,Truncation,NaN
1418,Gallbladder Carcinoma,6.13,Female,Unknown,9,CDKN2A,None,frameshift_variant,04/11,51-60,None,NaN,None,Substitution/Indel,0.166667
1066,Extrahepatic Cholangiocarcinoma,3.00,Male,Unknown,9,CDKN2A,C>T,missense_variant,02/03,61-70,25398284.0-25398284.0,12.0,gGt/gAt,Truncation,0.166667
1673,Gastric Cancer,3.00,Male,Unknown,19,CIC,C>-,frameshift_variant,01/20,None,56435161.0-56435161.0,659.0,gGt/gt,Truncation,NaN
3316,Soft Tissue Sarcoma,9.40,None,Unknown,3,CTNNB1,A>G,missense_variant,03/15,None,41266124.0-41266124.0,41.0,Acc/Gcc,Substitution/Indel,NaN
2289,Liver Hepatocellular Carcinoma,6.10,Male,Unknown,3,CTNNB1,C>A,missense_variant,03/15,41-50,7577534.0-7577534.0,33.0,agG/agT,Substitution/Indel,NaN


In [45]:
merged_df = merged_df[merged_df['Lift Value'] > 1]
merged_df['Smoke Status'] = merged_df['Smoke Status'].replace('Unknown', np.nan)

In [46]:
# def row_to_sentence(row):
#     return " AND ".join(
#         f"{col} value is {val}"
#         for col, val in row.items()
#         if pd.notnull(val) and col != "Cancer Type" and col!="Lift Value"
#     )
def row_to_sentence(row):
    return " AND ".join(
        f"{col} value is {val}"
        for col, val in row.items()
        if pd.notnull(val) and col not in {"Cancer Type", "Lift Value", "hypo_factors"}
    )



In [47]:
merged_df['hypo_factors'] = merged_df.apply(row_to_sentence, axis=1)
merged_df.rename(columns={'Lift Value': 'support'}, inplace=True)
merged_df

,Cancer Type,support,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous),hypo_factors
118,Breast Carcinoma,3.03,Female,NaN,X,MED12,G>A,missense_variant,07/11,61-70,178936091.0-178936091.0,545.0,Gag/Aag,Substitution/Indel,0.166667,Sex value is Female AND Chromosome value is X ...
193,Breast Carcinoma,6.15,Female,NaN,17,TP53,G>A,missense_variant,08/11,51-60,178936091.0-178936091.0,545.0,Gag/Aag,Substitution/Indel,0.133333,Sex value is Female AND Chromosome value is 17...
219,Breast Carcinoma,9.34,Female,NaN,2,ALK,C>T,missense_variant,10/21,41-50,None,542.0,None,Substitution/Indel,0.133333,Sex value is Female AND Chromosome value is 2 ...
230,Breast Carcinoma,14.78,Female,NaN,3,PIK3CA,A>G,missense_variant,21/21,41-50,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,0.100000,Sex value is Female AND Chromosome value is 3 ...
236,Breast Carcinoma,22.01,Female,NaN,3,PIK3CA,A>G,missense_variant,21/21,None,None,NaN,None,Substitution/Indel,NaN,Sex value is Female AND Chromosome value is 3 ...
238,Breast Carcinoma,26.22,Female,NaN,3,PIK3CA,A>G,missense_variant,21/21,None,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,NaN,Sex value is Female AND Chromosome value is 3 ...
472,Colorectal Carcinoma,3.00,Female,NaN,5,APC,C>T,stop_gained,16/16,51-60,112175639.0-112175639.0,1450.0,Cga/Tga,Truncation,NaN,Sex value is Female AND Chromosome value is 5 ...
751,Esophageal Carcinoma,3.02,Male,NaN,17,TP53,C>T,stop_gained,08/11,51-60,7577538.0-7577538.0,248.0,cGg/cAg,Truncation,0.266667,Sex value is Male AND Chromosome value is 17 A...
814,Esophageal Carcinoma,6.08,Male,NaN,9,NOTCH1,C>T,missense_variant,02/05,None,None,NaN,None,Substitution/Indel,NaN,Sex value is Male AND Chromosome value is 9 AN...
820,Esophageal Carcinoma,10.73,Male,NaN,9,NOTCH1,None,None,None,None,None,NaN,None,Truncation,NaN,Sex value is Male AND Chromosome value is 9 AN...


In [49]:
merged_df.columns

Index(['Cancer Type', 'support', 'Sex', 'Smoke Status', 'Chromosome',
       'Hugo_Symbol', 'SNP_event', 'Consequence', 'Exon_Number',
       'Diagnosis Age', 'Position', 'Protein_position', 'Codons',
       'VAR_TYPE_SX', 'TMB (nonsynonymous)', 'hypo_factors'],
      dtype='object')

In [50]:
merged_df.to_csv("models_hypotheses/LIFT_hypotheses_as_sentences.csv")

In [51]:
merged_df.to_csv("data_for_lift_merged.csv")

In [35]:
lift_merged = pd.read_csv("lifts_merged.csv", index_col=0)

In [36]:
lift_merged.sort_values(by="Lift Value", ascending=False)

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '12', 'KRAS', 'C>G', 'missense_var...",27.03
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'A>G', 'm...",26.22
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'A>G', 'm...",26.22
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'missense...",23.73
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', '21/21', ...",23.73
...,...,...,...
Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '4', 'FAT1', 'Truncation')",0.00
Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '16', 'PLCG2', 'Substituti...",0.00
Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Smoker', '8', 'FAM135B', 'Substituti...",0.00


In [37]:
df = lift_merged.copy()
df.reset_index(inplace=True)

In [39]:
df[df["Lift Value"] == 27.03]

,Cancer Type,Feature Combination,Feature,Lift Value
12652,Pancreatic Cancer,"(Smoke Status, Chromosome, Hugo_Symbol, SNP_ev...","(Unknown, 12, KRAS, C>G, missense_variant, 02/...",27.03
